In [36]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import math
from scipy.misc import imsave
from scipy.misc import imread
import pandas as pd
import glob, os
################################################
# Read frames which have been data-cleaned (wire_particle_tracking_datacleaning). 
################################################
shot = 120
picL = 18333
picR = 18158
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
TR = [[-5.7704E-08,1.1998E-05,-2.1514E-03],[8.9694E-06,5.4087E-07,-3.9328E-02],[-1.7813E-03,3.3894E-02,9.9865E-01]]
TL = [[1.1040E-07,2.2954E-05,-4.1584E-03],[1.7160E-05,-1.0348E-06,6.8654E-02],[-3.4505E-03,-7.3661E-02,9.9490E-01]]
BL = [[-1.0175E-07,2.1155E-05,-4.2910E-03],[1.5815E-05,9.5369E-07,6.2907E-02],[-2.8931E-03,-6.8254E-02,9.9567E-01]]
F_10point = [[  8.34360564e-08,   1.89022714e-05,  -3.73041652e-03],[  1.94501963e-05,   1.19512678e-06,  -5.90813947e-02],[ -3.16737708e-03,   5.30589622e-02,   9.96830069e-01]]
F_big = [[  1.75345969e-07,   1.87353876e-05,  -3.69560210e-03], [  1.92508173e-05,   9.79548859e-07,  -5.72239187e-02],[ -3.15390413e-03,   5.12220927e-02,   9.97034661e-01]]
F = F_big
Matr = F
MatrixL = np.array(Matr)
MatrixR = MatrixL.transpose()
Linex = np.arange(0,384,1)
v0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL_sum.tif'.format(shot,shot,picL))
v0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR_sum.tif'.format(shot,shot,picR))
bk0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL0.tif'.format(shot,shot,picL))
bk0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR0.tif'.format(shot,shot,picR))

In [37]:
#pair = pd.read_csv("PairList.csv") 
#for iL in pair['PIDL'].unique():

#    for iR in pair1['PIDR'].unique():

In [38]:
#os.chdir(".")
#results = pd.DataFrame([])
for counter, file in enumerate(glob.glob("./Data/Shot{}/Pair_*.csv".format(shot))):    
    f1 = file.split('.csv')
    f2 = f1[0].split('_')    
    df = pd.read_csv(file)
    plt.figure(figsize = (24,24))  # make a new figure
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    ax2.imshow(v0L+bk0L)
    ax2.scatter(df['XL'],df['YL'],color='r',marker=".",s=1)
    #ax2.plot(df['XR'],df['YR'],color='y')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax4.imshow(v0R+bk0R)
    #ax4.plot(df['XL'],df['YL'],color='r')
    ax4.scatter(df['XR'],df['YR'],color='y',marker=".",s=1)
    ax4.set_xlabel('x')
    ax4.set_ylabel('y')
    ax1.scatter(df['Frame'],df['MassL'],color='r',marker=".",s=1)
    ax1.scatter(df['Frame'],df['MassR'],color='y',marker=".",s=1)
    ax1.set_xlabel('Frame')
    ax1.set_ylabel('Mass')
    ax3.scatter(df['Frame'],df['DistanceL'],color='r',marker=".",s=1)
    ax3.scatter(df['Frame'],df['DistanceR'],color='y',marker=".",s=1)
    ax3.set_xlabel('Frame')
    ax3.set_ylabel('FF')
    plt.savefig('./Data/Shot{}/Pair_{}.pdf'.format(shot,f2[1]))
